## Cralwer dos sites de TI

In [3]:
#Instalando o bs4
!pip install beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 22.3 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


### Imports

In [129]:
import math, sys, os

#Datas
import pandas as pd
import numpy as np

#Web
from bs4 import BeautifulSoup
import requests

#Plotters
import matplotlib as plt
import seaborn as sns

#Text
import re
import json

#handling dates
from datetime import datetime

%load_ext autoreload
%autoreload 2

%time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


### Setando Header

In [5]:
header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" ,
    'referer':'https://www.google.com/'
}

### Criando a função da Pichau

In [98]:
def get_html_page(url : str, header=None) -> str:

    '''
    Funcao para o script HTML das paginas a serem monitoradas

    Retorno do metodo
    str

    Parametros

    url:str
    link do site a ser obtidos o script
    '''

    #Checa se existe o header
    if header is None  :
        header = {
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" ,
            'referer':'https://www.google.com/'
        }

    #Checa se a url está vazia antes de requisitar
    if len(url) == 0:
        print('URL vazia')
        return None

    r = requests.get(url, headers=header)
    
    return_code = r.status_code

    if not return_code in (200, 201, 202):
        print(f'Houve um erro no processamento da url {url}')
        return None
    
    return r.text


In [99]:
res = get_html_page('https://www.pichau.com.br/processador-amd-ryzen-5-5600-6-core-12-threads-3-5ghz-4-4ghz-turbo-cache-35mb-am4-100-100000927box?utm_source=pcbuildwizard')

In [169]:
def extract_pichau(html_script : str) -> pd.DataFrame:

    '''
    Funcao para buscar as informacoes do site da pichau e persisti-las

    Retorno do metodo
    pandas.DataFrame

    Parametros

    html_script:str
    script do site a ser obtidos as informações
    '''

    limite_galeria = 6

    df_retorno = pd.DataFrame()

    soup = BeautifulSoup(html_doc, 'html.parser')

    all_data = soup.find('script', {'type':'application/json'})

    jsn = json.loads(all_data.string)

    #Usar esta lista de atributos neste nível: ['props']['pageProps']['pageData']['content']
    lista_attr_l0 = ['potencia', 'name', 'stock_status', 'codigo_barra', 'is_openbox']

    for i in lista_attr_l0:
        df_retorno[i] = pd.Series(jsn['props']['pageProps']['pageData']['content'][i])

    #Usar esta lista de atributos neste nível: ['props']['pageProps']['pageData']['content']['mysales_promotion']
    lista_attr_l1 = ['promotion_name', 'qty_available', 'qty_sold', 'expire_at']

    for i in lista_attr_l1:
        df_retorno[i] = pd.Series(jsn['props']['pageProps']['pageData']['content']['mysales_promotion'][i])

    galeria = jsn['props']['pageProps']['pageData']['content']['media_gallery']

    for img, counter in zip(galeria [:min(len(galeria), limite_galeria)], range(limite_galeria)):
        df_retorno["url_image" + str(counter)] = img['url']

    df_retorno['currency_promo'] = pd.Series(jsn['props']['pageProps']['pageData']['content']['price_range']['minimum_price']['final_price']['currency'])
    df_retorno['value_promo'] = pd.Series(jsn['props']['pageProps']['pageData']['content']['price_range']['minimum_price']['final_price']['value'])

    df_retorno['currency_regular'] = pd.Series(jsn['props']['pageProps']['pageData']['content']['price_range']['minimum_price']['regular_price']['currency'])
    df_retorno['value_regular'] = pd.Series(jsn['props']['pageProps']['pageData']['content']['price_range']['minimum_price']['regular_price']['value'])

    df_retorno['currency_cash'] = df_retorno['currency_promo']
    df_retorno['value_cash'] = df_retorno['value_promo'].astype('float64') * 0.88

    df_retorno['dt_hr_captura'] = [datetime.now()]
    print(df_retorno.shape)

    return df_retorno
    #data hora captura


In [170]:
df = extract_pichau(res)

(1, 18)


In [171]:
df

,potencia,name,stock_status,codigo_barra,is_openbox,promotion_name,qty_available,qty_sold,expire_at,url_image0,url_image1,currency_promo,value_promo,currency_regular,value_regular,currency_cash,value_cash,dt_hr_captura
0,65,"Processador AMD Ryzen 5 5600, 6-Core, 12-Threa...",IN_STOCK,0730143314190,0,Ferias com o Pombo,83,264,2022-08-01 17:00:00,https://media.pichau.com.br/media/catalog/prod...,https://media.pichau.com.br/media/catalog/prod...,BRL,1269.32,BRL,1697.3,BRL,1117.0016,2022-07-21 22:00:27.586661


In [145]:
#Usar esta lista de atributos neste nível: ['props']['pageProps']['pageData']['content']
lista_attr_l0 = ['potencia', 'name', 'stock_status', 'codigo_barra', 'is_openbox']

In [122]:
for i in lista_attr_l0:
    print(jsn['props']['pageProps']['pageData']['content'][i])

65
Processador AMD Ryzen 5 5600, 6-Core, 12-Threads, 3.5GHz (4.4GHz Turbo), Cache 35MB, AM4, 100-100000927BOX
IN_STOCK
0730143314190
0


In [119]:
#Usar esta lista de atributos neste nível: ['props']['pageProps']['pageData']['content']['mysales_promotion']
lista_attr_l1 = ['promotion_name', 'qty_available', 'qty_sold', 'expire_at']

In [120]:
for i in lista_attr_l1:
    print(jsn['props']['pageProps']['pageData']['content']['mysales_promotion'][i])

Ferias com o Pombo
83
264
2022-08-01 17:00:00


In [111]:
jsn['props']['pageProps']['pageData']['content']['mysales_promotion']

{'expire_at': '2022-08-01 17:00:00',
 'price_discount': 25.22,
 'price_promotional': 1269.32,
 'promotion_name': 'Ferias com o Pombo',
 'promotion_url': 'promocao/feriaspombo',
 'qty_available': 83,
 'qty_sold': 264,
 '__typename': 'MySalesPromotionProduct'}

In [ ]:
#moeda: ['props']['pageProps']['pageData']['content']['price_range']['minimum_price']['final_price']['currency']
#value_promocional: ['props']['pageProps']['pageData']['content']['price_range']['minimum_price']['final_price']['value']

#moeda: jsn['props']['pageProps']['pageData']['content']['price_range']['minimum_price']['regular_price']['currency']
#value_regular: jsn['props']['pageProps']['pageData']['content']['price_range']['minimum_price']['regular_price']['value']


In [71]:
galeria = jsn['props']['pageProps']['pageData']['content']['media_gallery']]
limit = 6
for i in galeria [:min(len(galeria), limit)]:
    print(i['url'])

https://media.pichau.com.br/media/catalog/product/cache/2f958555330323e505eba7ce930bdf27/1/0/100-100000927box_1.jpg
https://media.pichau.com.br/media/catalog/product/cache/2f958555330323e505eba7ce930bdf27/1/0/100-100000927box1_1.jpg


In [74]:
jsn['props']['pageProps']['pageData']['content']

{'id': 31200,
 'sku': '100-100000927BOX',
 'url_key': 'processador-amd-ryzen-5-5600-6-core-12-threads-3-5ghz-4-4ghz-turbo-cache-35mb-am4-100-100000927box',
 'name': 'Processador AMD Ryzen 5 5600, 6-Core, 12-Threads, 3.5GHz (4.4GHz Turbo), Cache 35MB, AM4, 100-100000927BOX',
 'special_price': 1269.32,
 'socket': '539',
 'hide_from_search': 0,
 'is_openbox': 0,
 'openbox_state': None,
 'openbox_condition': None,
 'tipo_de_memoria': None,
 'caracteristicas': None,
 'slots_memoria': None,
 'marcas': 116,
 'marcas_info': {'name': 'AMD', '__typename': 'MarcasProduct'},
 'formato_placa': None,
 'plataforma': '7',
 'portas_sata': None,
 'slot_cooler_120': None,
 'slot_cooler_80': None,
 'slot_cooler_140': None,
 'slot_cooler_200': None,
 'coolerbox_included': 1,
 'potencia': '65',
 'quantidade_pacote': None,
 'alerta_monteseupc': None,
 'vgaintegrado': 153,
 'product_set_name': 'Processador',
 'categories': [{'name': 'Hardware', '__typename': 'CategoryTree'},
  {'name': 'Processadores', '__typ

In [116]:
#teste url
r_teste = requests.get('https://www.kabum.com.br/produto/320798/processador-amd-ryzen-5-5600-cache-35mb-3-5ghz-4-4ghz-max-turbo-am4-sem-video-100-100000927box', headers=header)
r_teste.status_code

html_doc_teste = r_teste.text
soup_teste = BeautifulSoup(html_doc_teste, 'html.parser')
all_data_teste = soup_teste.find('script', {'type':'application/json'})
jsn_teste = json.loads(all_data_teste.string)
jsn_teste



{'props': {'pageProps': {'flagBlackFriday': False,
   'data': {'offerBannerImage': 'https://themes.kabum.com.br/conteudo/oferta/108/banner/julho-gamer_descricao.jpg',
    'productCatalog': '{"code":320798,"productSpecie":0,"name":"Processador AMD Ryzen 5 5600, Cache 35MB, 3.5GHz (4.4GHz Max Turbo), AM4, Sem Vídeo - 100-100000927BOX","description":"<h2>Processador AMD Ryzen 5 5600</h2>\\n<p>&nbsp;</p>\\n<p>Quando voc&ecirc; tem a arquitetura de processador&nbsp;de desktop&nbsp;mais avan&ccedil;ada do mundo2&nbsp;para jogadores e criadores de conte&uacute;do, as possibilidades s&atilde;o infinitas. Esteja voc&ecirc; jogando os jogos mais recentes, projetando o pr&oacute;ximo pr&eacute;dio ou processando dados, voc&ecirc; precisa de um processador poderoso que possa lidar com tudo - e muito mais. Sem d&uacute;vida, os processadores AMD Ryzen &trade; s&eacute;rie 5000 definem o padr&atilde;o para jogadores e artistas.</p>\\n<p>&nbsp;</p>\\n<h2>A ferramenta certa para o trabalho</h2>\\n<div

In [ ]:
aaaaa

In [176]:
def extract_pichau(html_script : str) -> pd.DataFrame:

    '''
    Funcao para buscar as informacoes do site da pichau e persisti-las

    Retorno do metodo
    pandas.DataFrame

    Parametros

    html_script:str
    script do site a ser obtidos as informações
    '''

    limite_galeria = 6

    df_retorno1 = pd.DataFrame()

    soup = BeautifulSoup(html_doc, 'html.parser')

    all_data = soup.find('script', {'type':'application/json'})

    jsn = json.loads(all_data.string)

    #Usar esta lista de atributos neste nível: ['props']['pageProps']['pageData']['content']
    lista_attr_l0 = ['potencia', 'name', 'stock_status', 'codigo_barra', 'is_openbox']

    for i in lista_attr_l0:
        df_retorno1[i] = jsn['props']['pageProps']['pageData']['content'][i]

    #Usar esta lista de atributos neste nível: ['props']['pageProps']['pageData']['content']['mysales_promotion']
    lista_attr_l1 = ['promotion_name', 'qty_available', 'qty_sold', 'expire_at']

    for i in lista_attr_l1:
        df_retorno1[i] = jsn['props']['pageProps']['pageData']['content']['mysales_promotion'][i]

    galeria = jsn['props']['pageProps']['pageData']['content']['media_gallery']

    for img, counter in zip(galeria [:min(len(galeria), limite_galeria)], range(limite_galeria)):
        df_retorno1["url_image" + str(counter)] = img['url']

    df_retorno1['currency_promo'] = jsn['props']['pageProps']['pageData']['content']['price_range']['minimum_price']['final_price']['currency']
    df_retorno1['value_promo'] = jsn['props']['pageProps']['pageData']['content']['price_range']['minimum_price']['final_price']['value']

    df_retorno1['currency_regular'] = jsn['props']['pageProps']['pageData']['content']['price_range']['minimum_price']['regular_price']['currency']
    df_retorno1['value_regular'] = jsn['props']['pageProps']['pageData']['content']['price_range']['minimum_price']['regular_price']['value']

    df_retorno1['currency_cash'] = df_retorno['currency_promo']
    value_cash = df_retorno1['value_promo'].astype('float64') * 0.88
    value_cash = [value_cash]
    df_retorno1['value_cash'] = value_cash
    df_retorno1['dt_hr_captura'] = [datetime.now()]
    print(df_retorno1.shape)

    return df_retorno1
    #data hora captura
df_retorno1

In [177]:
df_retorno1

NameError: name 'df_retorno1' is not defined

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b8b98786-19ce-460b-98f4-40a20facf290' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>